In [1]:
import os
import shutil

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
print(torch.cuda.is_available())

from ultralytics import YOLO

np.random.seed(42)

True


In [11]:
#move labels to corresponding location
for env in ["snow", "dessert"]:
    for train_test_val in ["train", "test", "valid"]:
        for f in os.listdir(f"./data/bias/{env}/{train_test_val}/images"):
            f_name = f[:-3]+"txt"
            shutil.copy(f"./data/raw/labels/{f_name}", f"./data/bias/{env}/{train_test_val}/labels/{f_name}")

In [2]:
DATA_PATH = "E:\\OU\\master\\rm\\pre_shot_sniper_detection\\data\\data_aug_bias_dessert.yaml"
model = YOLO("./runs/weights_normal/350_imgs/weights/best.pt")

In [3]:
results_dessert = model.val(data=DATA_PATH)
results_dessert.results_dict

Ultralytics YOLOv8.0.200  Python-3.9.18 torch-2.1.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 10239MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning E:\OU\master\rm\pre_shot_sniper_detection\data\bias\dessert\valid\labels... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<00:00, 1044.73it/s]
val: New cache created: E:\OU\master\rm\pre_shot_sniper_detection\data\bias\dessert\valid\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.19s/it]
                   all         46         53       0.78      0.566      0.607      0.332
Speed: 2.0ms preprocess, 8.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\val10


{'metrics/precision(B)': 0.7804080020807009,
 'metrics/recall(B)': 0.5660377358490566,
 'metrics/mAP50(B)': 0.6074543304671212,
 'metrics/mAP50-95(B)': 0.3317384728136813,
 'fitness': 0.3593100585790253}

In [4]:
DATA_PATH = "E:\\OU\\master\\rm\\pre_shot_sniper_detection\\data\\data_aug_bias_snow.yaml"
model = YOLO("./runs/weights_normal/350_imgs/weights/best.pt")

In [5]:
results_snow = model.val(data=DATA_PATH)
results_snow.results_dict

Ultralytics YOLOv8.0.200  Python-3.9.18 torch-2.1.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 10239MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning E:\OU\master\rm\pre_shot_sniper_detection\data\bias\snow\valid\labels... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<00:00, 908.09it/s]
val: New cache created: E:\OU\master\rm\pre_shot_sniper_detection\data\bias\snow\valid\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:04<00:00,  1.37s/it]
                   all         46         53       0.76      0.566      0.564      0.306
Speed: 6.4ms preprocess, 2.1ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs\detect\val11


{'metrics/precision(B)': 0.7595231903749659,
 'metrics/recall(B)': 0.5660377358490566,
 'metrics/mAP50(B)': 0.5635075152279464,
 'metrics/mAP50-95(B)': 0.30575799343264776,
 'fitness': 0.3315329456121776}

In [ ]:
results_snow = model.val(data="./data/bias/snow/")
results_snow.results_dict